In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from ipywidgets import interact

In [2]:
# Load your data
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [ ]:
# Prepare item names for the combobox
item_names = sorted(df['type_name'].dropna().astype(str).unique())
combobox = widgets.Combobox(
    options=item_names,
    placeholder='Search item',
    description='Item:',
    ensure_option=True,
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)
display(combobox)

Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, layout=Layout(width='400p…

In [ ]:
def predict_and_plot(selected_item):
    # Filter and sort
    df_item = df[df['type_name'] == selected_item].copy()
    df_item = df_item.sort_values('date').reset_index(drop=True)
    df_item['next_avg'] = df_item['average'].shift(-1)
    df_item['delta'] = df_item['next_avg'] - df_item['average']
    df_item = df_item.dropna(subset=['next_avg', 'delta'])

    if len(df_item) < 10:
        print("Not enough data for this item!")
        return

    features = ['average', 'highest', 'lowest', 'volume']
    X = df_item[features]
    y_price = df_item['next_avg']
    y_delta = df_item['delta']

    split_idx = int(len(df_item) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_price_train, y_price_test = y_price.iloc[:split_idx], y_price.iloc[split_idx:]
    y_delta_train, y_delta_test = y_delta.iloc[:split_idx], y_delta.iloc[split_idx:]

    # Linear Regression for next day's price
    model_price = LinearRegression()
    model_price.fit(X_train, y_price_train)
    y_price_pred = model_price.predict(X_test)

    # Linear Regression for delta (change)
    model_delta = LinearRegression()
    model_delta.fit(X_train, y_delta_train)
    y_delta_pred = model_delta.predict(X_test)

    # Naive baseline: tomorrow = today
    y_naive = X_test['average'].values

    # Metrics for price prediction
    price_mae = mean_absolute_error(y_price_test, y_price_pred)
    price_r2 = r2_score(y_price_test, y_price_pred)
    naive_mae = mean_absolute_error(y_price_test, y_naive)
    naive_r2 = r2_score(y_price_test, y_naive)

    # Metrics for delta prediction (price change)
    delta_mae = mean_absolute_error(y_delta_test, y_delta_pred)
    delta_r2 = r2_score(y_delta_test, y_delta_pred)
    naive_delta = [0] * len(y_delta_test)  # Naive prediction: no change
    naive_delta_mae = mean_absolute_error(y_delta_test, naive_delta)
    naive_delta_r2 = r2_score(y_delta_test, naive_delta)

    # Plot price prediction
    plt.figure(figsize=(14, 5))
    plt.plot(y_price_test.index, y_price_test, label='Actual Next Avg Price', marker='o')
    plt.plot(y_price_test.index, y_price_pred, label='Predicted Next Avg Price', marker='x')
    plt.plot(y_price_test.index, y_naive, label='Naive Prediction (today)', linestyle='dashed', marker='.')
    plt.xlabel('Test Set Index')
    plt.ylabel('Average Price')
    plt.title(f'Next Day Price Prediction: {selected_item}')
    plt.legend()
    plt.show()

    print(f"\033[1mNext-Day Price Prediction (Regression)\033[0m")
    print(f"Mean Absolute Error: {price_mae:.2f}")
    print(f"R2 Score: {price_r2:.4f}")
    print(f"\n\033[1mNaive Baseline (Next=Today)\033[0m")
    print(f"Mean Absolute Error: {naive_mae:.2f}")
    print(f"R2 Score: {naive_r2:.4f}")

    # Plot delta prediction
    plt.figure(figsize=(14, 5))
    plt.plot(y_delta_test.index, y_delta_test, label='Actual Delta', marker='o')
    plt.plot(y_delta_test.index, y_delta_pred, label='Predicted Delta', marker='x')
    plt.plot(y_delta_test.index, naive_delta, label='Naive Prediction (no change)', linestyle='dashed', marker='.')
    plt.xlabel('Test Set Index')
    plt.ylabel('Price Change (Delta)')
    plt.title(f'Next Day Price Change Prediction (Delta): {selected_item}')
    plt.legend()
    plt.show()

    print(f"\n\033[1mNext-Day Delta Prediction (Regression)\033[0m")
    print(f"Mean Absolute Error: {delta_mae:.2f}")
    print(f"R2 Score: {delta_r2:.4f}")
    print(f"\n\033[1mNaive Baseline (Delta=0)\033[0m")
    print(f"Mean Absolute Error: {naive_delta_mae:.2f}")
    print(f"R2 Score: {naive_delta_r2:.4f}")

    # Comparison table for first 10 test predictions
    comp = pd.DataFrame({
        'Actual Price': y_price_test.values,
        'Pred Price': y_price_pred,
        'Naive': y_naive,
        'Actual Delta': y_delta_test.values,
        'Pred Delta': y_delta_pred,
        'Naive Delta': naive_delta
    }, index=y_price_test.index)
    display(comp.head(10))

# Interactive selection
interact(predict_and_plot, selected_item=combobox)

interactive(children=(Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, lay…

<function __main__.predict_and_plot(selected_item)>